#### Create word embedding from corpus by Pretrained News vectors

In [3]:
from __future__ import unicode_literals, print_function
import gensim, logging, os
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import tokenize
import string
from itertools import chain

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
#word2vec.FAST_VERSION

In [5]:
content_dir = '../../Data/input/content'

class Corpus_Sentences(object):
    def __init__(self, dirname):
        self.dirname = dirname;
        
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()
                
sentences = Corpus_Sentences(content_dir) # a memory-friendly iterator

In [6]:
glove_wv = KeyedVectors.load_word2vec_format('../../../../pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt', binary=False)

2018-04-26 04:58:30,982 : INFO : loading projection weights from ../../../../pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt
2018-04-26 05:00:33,884 : INFO : loaded (400000, 300) matrix from ../../../../pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt


In [7]:
model = Word2Vec(size=300, min_count=1, iter=10)
model.build_vocab(sentences)
training_examples_count = model.corpus_count
# below line will make it 1, so saving it before
model.build_vocab([list(glove_wv.vocab.keys())], update=True)

2018-04-26 05:00:33,893 : INFO : collecting all words and their counts
2018-04-26 05:00:34,062 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-26 05:00:44,314 : INFO : collected 22787 word types from a corpus of 485731 raw words and 1305 sentences
2018-04-26 05:00:44,315 : INFO : Loading a fresh vocabulary
2018-04-26 05:00:44,559 : INFO : min_count=1 retains 22787 unique words (100% of original 22787, drops 0)
2018-04-26 05:00:44,560 : INFO : min_count=1 leaves 485731 word corpus (100% of original 485731, drops 0)
2018-04-26 05:00:44,666 : INFO : deleting the raw counts dictionary of 22787 items
2018-04-26 05:00:44,668 : INFO : sample=0.001 downsamples 39 most-common words
2018-04-26 05:00:44,669 : INFO : downsampling leaves estimated 353695 word corpus (72.8% of prior 485731)
2018-04-26 05:00:44,752 : INFO : estimated required memory for 22787 words and 300 dimensions: 66082300 bytes
2018-04-26 05:00:44,753 : INFO : resetting layer weights
2018-04-26

In [8]:
#if lockf=0.0 then pretrained word vectors will not be trained so keeping lockf=1.0

model.intersect_word2vec_format("../../../../pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt",binary=False, lockf=1.0)

2018-04-26 05:01:00,959 : INFO : loading projection weights from ../../../../pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt
2018-04-26 05:03:19,880 : INFO : merged 400000 vectors into (410894, 300) matrix from ../../../../pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt


In [9]:
model.train(sentences, total_examples=training_examples_count, epochs=100)

2018-04-26 05:03:19,886 : INFO : training model with 3 workers on 410894 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2018-04-26 05:03:21,011 : INFO : EPOCH 1 - PROGRESS: at 8.05% examples, 26270 words/s, in_qsize 0, out_qsize 0
2018-04-26 05:03:22,101 : INFO : EPOCH 1 - PROGRESS: at 18.08% examples, 31060 words/s, in_qsize 0, out_qsize 0
2018-04-26 05:03:23,307 : INFO : EPOCH 1 - PROGRESS: at 29.35% examples, 31434 words/s, in_qsize 0, out_qsize 0
2018-04-26 05:03:24,332 : INFO : EPOCH 1 - PROGRESS: at 41.23% examples, 32884 words/s, in_qsize 0, out_qsize 0
2018-04-26 05:03:25,371 : INFO : EPOCH 1 - PROGRESS: at 52.18% examples, 35671 words/s, in_qsize 0, out_qsize 0
2018-04-26 05:03:26,582 : INFO : EPOCH 1 - PROGRESS: at 62.53% examples, 37695 words/s, in_qsize 0, out_qsize 0
2018-04-26 05:03:27,727 : INFO : EPOCH 1 - PROGRESS: at 72.64% examples, 39628 words/s, in_qsize 0, out_qsize 0
2018-04-26 05:03:28,864 : INFO : EPOCH 1 - PROGRESS: at 83.37% exa

2018-04-26 05:03:50,541 : INFO : EPOCH - 14 : training on 485731 raw words (483047 effective words) took 1.5s, 319262 effective words/s
2018-04-26 05:03:51,558 : INFO : EPOCH 15 - PROGRESS: at 76.40% examples, 325486 words/s, in_qsize 4, out_qsize 1
2018-04-26 05:03:51,917 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-26 05:03:51,920 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-26 05:03:51,943 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-26 05:03:51,944 : INFO : EPOCH - 15 : training on 485731 raw words (483015 effective words) took 1.4s, 345638 effective words/s
2018-04-26 05:03:53,000 : INFO : EPOCH 16 - PROGRESS: at 78.01% examples, 322347 words/s, in_qsize 5, out_qsize 0
2018-04-26 05:03:53,336 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-26 05:03:53,339 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-26 05:03:53,345 : INFO : worker threa

2018-04-26 05:04:12,787 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-26 05:04:12,788 : INFO : EPOCH - 30 : training on 485731 raw words (483070 effective words) took 1.4s, 348561 effective words/s
2018-04-26 05:04:13,821 : INFO : EPOCH 31 - PROGRESS: at 79.69% examples, 338065 words/s, in_qsize 5, out_qsize 0
2018-04-26 05:04:14,146 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-26 05:04:14,147 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-26 05:04:14,148 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-26 05:04:14,149 : INFO : EPOCH - 31 : training on 485731 raw words (483032 effective words) took 1.4s, 355514 effective words/s
2018-04-26 05:04:15,204 : INFO : EPOCH 32 - PROGRESS: at 79.69% examples, 331787 words/s, in_qsize 5, out_qsize 0
2018-04-26 05:04:15,493 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-26 05:04:15,507 : INFO : worker threa

2018-04-26 05:04:34,750 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-26 05:04:34,767 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-26 05:04:34,768 : INFO : EPOCH - 46 : training on 485731 raw words (483015 effective words) took 1.4s, 352295 effective words/s
2018-04-26 05:04:35,786 : INFO : EPOCH 47 - PROGRESS: at 79.69% examples, 343804 words/s, in_qsize 4, out_qsize 0
2018-04-26 05:04:36,097 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-26 05:04:36,129 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-26 05:04:36,140 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-26 05:04:36,141 : INFO : EPOCH - 47 : training on 485731 raw words (483028 effective words) took 1.4s, 352909 effective words/s
2018-04-26 05:04:37,215 : INFO : EPOCH 48 - PROGRESS: at 66.05% examples, 252982 words/s, in_qsize 0, out_qsize 1
2018-04-26 05:04:37,903 : INFO : worker threa

2018-04-26 05:04:59,045 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-26 05:04:59,064 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-26 05:04:59,084 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-26 05:04:59,085 : INFO : EPOCH - 62 : training on 485731 raw words (483030 effective words) took 1.4s, 356043 effective words/s
2018-04-26 05:05:00,104 : INFO : EPOCH 63 - PROGRESS: at 78.01% examples, 333546 words/s, in_qsize 5, out_qsize 0
2018-04-26 05:05:00,432 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-26 05:05:00,433 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-26 05:05:00,459 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-26 05:05:00,460 : INFO : EPOCH - 63 : training on 485731 raw words (483052 effective words) took 1.4s, 351935 effective words/s
2018-04-26 05:05:01,474 : INFO : EPOCH 64 - PROGRESS: at 78.01% exam

2018-04-26 05:05:21,273 : INFO : EPOCH 78 - PROGRESS: at 76.48% examples, 325180 words/s, in_qsize 5, out_qsize 0
2018-04-26 05:05:21,613 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-26 05:05:21,624 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-26 05:05:21,652 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-26 05:05:21,653 : INFO : EPOCH - 78 : training on 485731 raw words (483056 effective words) took 1.4s, 346780 effective words/s
2018-04-26 05:05:22,660 : INFO : EPOCH 79 - PROGRESS: at 70.88% examples, 298924 words/s, in_qsize 3, out_qsize 0
2018-04-26 05:05:23,109 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-26 05:05:23,113 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-26 05:05:23,115 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-26 05:05:23,117 : INFO : EPOCH - 79 : training on 485731 raw words (483067 effecti

2018-04-26 05:05:43,942 : INFO : EPOCH - 93 : training on 485731 raw words (483065 effective words) took 1.4s, 343913 effective words/s
2018-04-26 05:05:44,974 : INFO : EPOCH 94 - PROGRESS: at 74.87% examples, 311362 words/s, in_qsize 5, out_qsize 0
2018-04-26 05:05:45,495 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-26 05:05:45,518 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-26 05:05:45,531 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-26 05:05:45,532 : INFO : EPOCH - 94 : training on 485731 raw words (483055 effective words) took 1.6s, 304980 effective words/s
2018-04-26 05:05:46,594 : INFO : EPOCH 95 - PROGRESS: at 70.96% examples, 283742 words/s, in_qsize 4, out_qsize 1
2018-04-26 05:05:47,013 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-26 05:05:47,022 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-26 05:05:47,036 : INFO : worker threa

(48304139, 48573100)

In [12]:
model.wv.save_word2vec_format('../../../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.bin')
model.wv.save_word2vec_format('../../../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt', binary=False)

2018-04-26 05:26:27,899 : INFO : storing 410894x300 projection weights into ../../../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.bin
